In [83]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
import json 
base_dir = 'dataset\\LBNL_FDD_Dataset_SDAHU\\'
import pickle
from sklearn.metrics import precision_recall_curve, auc,confusion_matrix
import json
from sklearn.model_selection import TimeSeriesSplit
import random

In [89]:
def load_data(file_name):
    df = pd.read_csv(base_dir+file_name, index_col='Datetime')
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    return df

def plot_line_graph(df1, df2, feature):
    plt.plot(df1[feature].values, 'g')
    plt.plot(df2[feature].values, 'r')
    plt.show()

import pandas as pd
from statsmodels.tsa.stattools import adfuller

def check_stationarity(data, column_name, significance_level=0.05):
    """
    Check the stationarity of a time series using the Augmented Dickey-Fuller (ADF) test.

    Parameters:
    - data: pd.DataFrame
        Input DataFrame with time series data.
    - column_name: str
        Name of the column containing the time series data.
    - significance_level: float, optional (default=0.05)
        Significance level for the ADF test.

    Returns:
    - bool
        True if the time series is stationary, False otherwise.
    """

    # Check if the specified column exists in the DataFrame
    if column_name not in data.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    try:
        # Perform the ADF test
        result = adfuller(data[column_name])

        # Extract ADF test statistic and p-value
        adf_statistic = result[0]
        p_value = result[1]
    
        # Compare p-value with significance level
        is_stationary = p_value <= significance_level
        
        if stationary:
            print(f"The time series is stationary. ADF Statistic: {adf_statistic}, p-value: {p_value}")
        else:
            print(f"The time series is not stationary. ADF Statistic: {adf_statistic}, p-value: {p_value}")
            
        #return is_stationary, adf_statistic, p_value
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

import pandas as pd
from sklearn.feature_selection import VarianceThreshold

def drop_low_variance_features(data, threshold=0.01):
    
    # Calculate variance of each feature
    variances = data.var()

    #print("Variances of features:")
    #print(variances)

    # Use VarianceThreshold to identify low variance features
    selector = VarianceThreshold(threshold)
    selector.fit(data)

    # Get indices of features to keep
    keep_indices = selector.get_support(indices=True)

    # Subset the DataFrame with selected features
    selected_data = data.iloc[:, keep_indices]

    return selected_data


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_classification_metrics(y_true, y_pred):
    """
    Evaluate classification metrics such as accuracy, precision, recall, and F1 score.

    Parameters:
    - y_test: list or array-like
        True labels.
    - y_pred: list or array-like
        Predicted labels.

    Returns:
    - dict
        Dictionary containing evaluation metrics.
    """
    # Initialize counts
    TP = TN = FP = FN = 0

    # Calculate confusion matrix elements
    for true, predicted in zip(y_true, y_pred):
        if true == 1 and predicted == 1:
            TP += 1
        elif true == 0 and predicted == 0:
            TN += 1
        elif true == 0 and predicted == 1:
            FP += 1
        elif true == 1 and predicted == 0:
            FN += 1

    print("TP:", TP)
    print("TN:", TN)
    print("FP:", FP)
    print("FN:", FN)

    # Compute classification metrics
    accuracy = round(accuracy_score(y_true, y_pred), 2)
    precision = round(precision_score(y_true, y_pred))
    recall = round(recall_score(y_true, y_pred), 2)
    f1 = round(f1_score(y_true, y_pred), 2)

    # Create a dictionary to store the metrics
    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

from sklearn.cluster import KMeans
def visualize_optimal_clusters(data, max_clusters=10):
    """
    Visualize the optimal number of clusters using the elbow method.

    Parameters:
    - data: array-like
        The training data for clustering.
    - max_clusters: int, optional (default=10)
        Maximum number of clusters to consider.

    Returns:
    None (plots the elbow curve).
    """

    distortions = []
    
    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(data)
        distortions.append(kmeans.inertia_)

    # Plotting the elbow curve
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, max_clusters + 1), distortions, marker='o')
    plt.title('Elbow Method for Optimal Number of Clusters')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Distortion (Inertia)')
    plt.show()



def predict_and_assign_labels(kmeans_model, test_data):
    """
    Predict cluster labels for test data and assign binary labels.

    Parameters:
    - kmeans_model: KMeans
        Trained KMeans model.
    - test_data: array-like
        Test data for prediction.

    Returns:
    - binary_labels: array-like
        Binary labels (0 or 1) assigned based on cluster membership.
    """

    # Predict cluster labels for the test data
    cluster_labels = kmeans_model.predict(test_data)
    print("cluster labels",cluster_labels) 
    # Assign binary labels (0 if in any cluster, 1 if not in any cluster)
    binary_labels = np.where(cluster_labels >= 0, 0, 1)

    return binary_labels

from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import GridSearchCV

def fine_tune_lof_model(non_faulty_data):
    """
    Fine-tune Local Outlier Factor (LOF) model on non-faulty data.

    Parameters:
    - non_faulty_data: pd.DataFrame or 2D array
        Non-faulty data for training the LOF model.

    Returns:
    - Trained LOF model.
    """

    # Fine-tune LOF model using GridSearchCV
    param_grid = {'n_neighbors': [5, 10, 15, 20], 'contamination': [0.01, 0.05, 0.1, 0.2]}
    lof = LocalOutlierFactor(novelty=True)
    grid_search = GridSearchCV(lof, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=True)
    grid_search.fit(non_faulty_data)

    # Train the best LOF model with optimal hyperparameters
    best_lof_model = LocalOutlierFactor(n_neighbors=grid_search.best_params_['n_neighbors'],
                                        contamination=grid_search.best_params_['contamination'], novelty=True)
    best_lof_model.fit(non_faulty_data)

    return best_lof_model

def evaluate_test_data(dict_fault, scaler, model, model_name, freq):
    
    results = {}
    df_predict_faults_vals = pd.DataFrame()
    df_hold_out_set_exp = pd.DataFrame()

    for file in dict_fault.keys():
        print(file)
        
        
        df_test = dict_faulty_df[file].loc[training_start_date_time : training_end_date_time].resample(freq).mean()


        df_hold_out_file = dict_faulty_df[file].loc[test_start_data:].resample(freq).mean()
        df_hold_out_set_exp = pd.concat([df_hold_out_set_exp, df_hold_out_file])

        
        test_fft = np.fft.fft(df_test)
        
        y_true = [1]*len(df_test)
        
        labels = trained_lof_model.predict(scaler.transform(test_fft.real))
        
        binary_labels = np.where(labels == -1, 1, 0)

        value_counts = Counter(binary_labels)

        #Print the counts
        for value, count in value_counts.items():
            print(f"{value}: {count} occurrences")
            
        metrics = evaluate_classification_metrics(y_true, binary_labels)
        print(metrics)
        results[file] = metrics
        print()

        fault_assignment = np.where(binary_labels == 1, True, False)
        
        fault_indices = np.where(fault_assignment)[0]
        
        df_faults_vals = df_test.iloc[fault_indices]
        
        df_predict_faults_vals = pd.concat([df_predict_faults_vals, df_faults_vals], ignore_index=True)
        
        
    with open('results/'+model_name+'_results'+str(freq)+'.json', "w") as outfile: 
        json.dump(results, outfile, indent=4)
        
    df_predict_faults_vals.to_csv('results/'+model_name+'_predicted_faults_vals'+'_sampling_freq_'+str(freq)+'.csv', sep=',', index=False, encoding='utf-8')

    df_hold_out_set_exp.to_csv('results/hold_out_test_set_explanations_sampling_freq_'+str(freq)+'.csv', sep=',', index=True, encoding='utf-8')

def load_fault_data(file_name):
    df = pd.read_csv(base_dir+file_name, index_col='Datetime')
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    return df


def cv_for_validation_eval(X, splits):
    tscv = TimeSeriesSplit(n_splits=splits)  # You can adjust the number of splits
    accuracy = []
    for train_index, valid_index in tscv.split(X):
    
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = [0]*len(X_train), [0]*len(X_valid)
    
        train_fft = np.fft.fft(X_train)
        scaler = MinMaxScaler()
        train_scaled_fft = scaler.fit_transform(train_fft.real)
        
        trained_lof_model = fine_tune_lof_model(train_scaled_fft)
    
        valid_fft = np.fft.fft(X_valid)
        
        valid_pred_labels = trained_lof_model.predict(scaler.transform(valid_fft.real))

        predicted_labels = np.where(valid_pred_labels == -1, 1, 0)
        
        counter_result = Counter(list(predicted_labels))
        print("Occurrences of items in NumPy array:")
        for item, count in counter_result.items():
            print(f"{item}: {count}")
        metrics = evaluate_classification_metrics(y_valid, predicted_labels)
        print(metrics)
        
        accuracy.append(metrics['Accuracy'])
        print()
    return np.mean(accuracy), np.std(accuracy)

def cv_for_test_set_eval(dict_fault, X, freq, splits):
    tscv = TimeSeriesSplit(n_splits=splits)  # You can adjust the number of splits
    results = {}
    counter = 1
    for file in dict_fault.keys():
        if 'short' in file:
            continue
        else:
            results[file] = {}
            results[file]['Accuracy'] = []
            results[file]['F1'] = []
            results[file]['Precision'] = []
            results[file]['Recall'] = []
            
    for train_index, test_index in tscv.split(X):
        
        X_train = X[train_index]
        train_fft = np.fft.fft(X_train)
        scaler = MinMaxScaler()
        train_scaled_fft = scaler.fit_transform(train_fft.real)
        trained_lof_model = fine_tune_lof_model(train_scaled_fft)

        for file in dict_fault.keys():
            if 'short' in file:
                continue
            else:
                df_test_resampled = dict_fault[file].resample(freq).mean()
               
                
                X_test = df_test_resampled.values[test_index]
                X_non_faulty = X[test_index]
                
                X_test_fault_non_fault = np.vstack((X_test, X_non_faulty))
                
                y_test = [1]*X_test.shape[0]
                y_test.extend([0]*X_non_faulty.shape[0])
                
                test_fft = np.fft.fft(X_test_fault_non_fault)
                test_pred_labels = trained_lof_model.predict(scaler.transform(test_fft.real))
                predicted_labels = np.where(test_pred_labels == -1, 1, 0)
                
                metrics = evaluate_classification_metrics(y_test, predicted_labels)
                print(counter, metrics)
                results[file]['Accuracy'].append(metrics['Accuracy'])
                results[file]['F1'].append(metrics['F1 Score'])
                results[file]['Precision'].append(metrics['Precision'])
                results[file]['Recall'].append(metrics['Recall'])
        print('*'*100)
        counter+=1
    return results

from sklearn.feature_selection import VarianceThreshold


def drop_low_variance_features(df, threshold=0.01):
    """
    Drop low-variance features from a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - threshold (float): Variance threshold below which features will be dropped.

    Returns:
    - pd.DataFrame: DataFrame with low-variance features dropped.
    - list: Names of dropped features.
    """

    # Initialize VarianceThreshold with the specified threshold
    selector = VarianceThreshold(threshold)

    # Fit the selector on the features
    selector.fit(df)

    # Get boolean mask indicating selected features
    selected_features = selector.get_support()

    # Get the names of the dropped features
    dropped_features = df.columns[~selected_features].tolist()

    # Drop low-variance features from the original DataFrame
    df_filtered = df.drop(columns=dropped_features)

    # Print information about dropped features
    if dropped_features:
        print(f"Dropped features with low variance: {', '.join(dropped_features)}")
    else:
        print("No features were dropped.")

    return df_filtered

def plot_features(df_correct, dict_fault, feature):

    for file in dict_fault.keys():
        print(file)
        plt.plot(df_correct[feature].values, 'g')
        plt.plot(dict_fault[file][feature].values, 'r')
        plt.show()
    

In [39]:
correct_data = load_data('AHU_annual.csv')

In [7]:
#df_correct_dropped_variance_features = drop_low_variance_features(correct_data)

Dropped features with low variance: OA_CFM, SA_SPSPT, SA_TEMPSPT, SF_SPD


In [8]:
#selected_features = list(df_correct_dropped_variance_features.keys())

In [40]:
dict_faulty_df = {}
for c, file in enumerate(os.listdir(base_dir)):
    if c>0:
        dict_faulty_df[file] = load_data(file)
        #dict_faulty_df[file] = df[selected_features]

In [ ]:
training_start_date_time = '2018-01-01 01:00:00'
training_end_date_time = '2018-10-31 23:59:00'

test_start_data = '2018-11-01 00:00:00'

In [90]:
resampling_freq = '5T'

In [91]:
df_train_resampled = correct_data.resample(resampling_freq).mean()
X = df_train_resampled.values

In [ ]:
test_set_cv_results = cv_for_test_set_eval(dict_faulty_df, X, resampling_freq, 10)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
TP: 6616
TN: 8883
FP: 672
FN: 2939
1 {'Accuracy': 0.81, 'Precision': 1, 'Recall': 0.69, 'F1 Score': 0.79}
TP: 5975
TN: 8883
FP: 672
FN: 3580
1 {'Accuracy': 0.78, 'Precision': 1, 'Recall': 0.63, 'F1 Score': 0.74}
TP: 6962
TN: 8883
FP: 672
FN: 2593
1 {'Accuracy': 0.83, 'Precision': 1, 'Recall': 0.73, 'F1 Score': 0.81}
TP: 6650
TN: 8883
FP: 672
FN: 2905
1 {'Accuracy': 0.81, 'Precision': 1, 'Recall': 0.7, 'F1 Score': 0.79}
TP: 6943
TN: 8883
FP: 672
FN: 2612
1 {'Accuracy': 0.83, 'Precision': 1, 'Recall': 0.73, 'F1 Score': 0.81}
TP: 6943
TN: 8883
FP: 672
FN: 2612
1 {'Accuracy': 0.83, 'Precision': 1, 'Recall': 0.73, 'F1 Score': 0.81}
TP: 6943
TN: 8883
FP: 672
FN: 2612
1 {'Accuracy': 0.83, 'Precision': 1, 'Recall': 0.73, 'F1 Score': 0.81}
TP: 6943
TN: 8883
FP: 672
FN: 2612
1 {'Accuracy': 0.83, 'Precision': 1, 'Recall': 0.73, 'F1 Score': 0.81}
TP: 5900
TN: 8883
FP: 672
FN: 3655
1 {'Accuracy': 0.77, 'Precision': 1, 'Recall': 0.62, 'F1 

In [ ]:
with open('results/10_fold_faults_non_faults_evaluation_features_dropped.json', "w") as outfile: 
        json.dump(test_set_cv_results, outfile, indent=4)

In [ ]:
import joblib

# Save the model to a file
joblib.dump(trained_lof_model, 'models/lof_model_'+str(resampling_freq)+'.pkl')
joblib.dump(scaler, 'models/minmax_model_'+str(resampling_freq)+'.pkl')